In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from wide_resnet import WideResnetLit
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import MLFlowLogger

L.seed_everything(42)
torch.set_float32_matmul_precision("medium")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
IMAGE_SIZE = 32
mean, std = [0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]
# source: https://pytorch.org/vision/stable/transforms.html
transforms_train = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)


transforms_test = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

train_dataset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transforms_train
)
test_dataset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transforms_test
)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=512, num_workers=31, persistent_workers=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=512, num_workers=31, persistent_workers=True
)
indices = torch.randperm(len(train_dataset))[: int(0.2 * len(train_dataset))]
validation_dataset = torch.utils.data.Subset(train_dataset, indices=indices)
validation_loader = torch.utils.data.DataLoader(
    dataset=validation_dataset, batch_size=512, num_workers=31, persistent_workers=True
)

In [ ]:
class DataModule(L.LightningDataModule):
    def __init__(self, train_loader, validation_loader):
        super().__init__()
        self.train_loader = train_loader
        self.validation_loader = validation_loader

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.validation_loader


data = DataModule(train_loader, validation_loader)
model = WideResnetLit(depth=32, weight_decay=10e-4)
logger = MLFlowLogger(experiment_name="WideResnet", save_dir="mlruns")
trainer = L.Trainer(
    max_epochs=50,
    logger=logger,
    callbacks=[
        ModelCheckpoint(
            monitor="val_acc",
            mode="max",
            dirpath="checkpoints/wide_resnet",
            filename="{epoch:02d}-{val_acc:.3f}",
        )
    ],
    precision="16-mixed",
    num_sanity_val_steps=0,
)
trainer.fit(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/dxzielinski/Desktop/github-repositories/optimization-data-analysis/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/dxzielinski/Desktop/github-repositories/optimization-data-analysis/models/checkpoints/wide_resnet exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | WideResNet         | 7.4 M  | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | test_acc  | MulticlassAccuracy | 0      | train
-------------------------------------------------

Epoch 0:   1%|          | 1/98 [00:01<02:05,  0.77it/s, v_num=55a6, train/loss_step=7.790, train/acc_step=0.109]

/home/dxzielinski/Desktop/github-repositories/optimization-data-analysis/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 0: 100%|██████████| 98/98 [00:26<00:00,  3.77it/s, v_num=55a6, train/loss_step=2.230, train/acc_step=0.238, val/loss=2.110, val/acc=0.194, train/loss_epoch=5.440, train/acc_epoch=0.157]

MisconfigurationException: `ModelCheckpoint(monitor='val_f1_macro')` could not find the monitored key in the returned metrics: ['train/loss', 'train/loss_step', 'train/acc', 'train/acc_step', 'val/loss', 'val/acc', 'train/loss_epoch', 'train/acc_epoch', 'epoch', 'step']. HINT: Did you call `log('val_f1_macro', value)` in the `LightningModule`?

Val F1 macro is 0.799

Val accuracy is 0.798

In [ ]:
best_ckpt = trainer.checkpoint_callback.best_model_path
print("Best checkpoint path:", best_ckpt)
trainer.test(model, datamodule=data, ckpt_path=best_ckpt)